<a href="https://colab.research.google.com/github/chacha86/20210227jsp/blob/master/%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%A7%91%EA%B0%92_%EC%98%88%EC%B8%A1_%EB%AC%B8%EC%A0%9C1_%ED%92%80%EC%9D%B43_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 로드
import pandas as pd
house = pd.read_csv('/content/1553768847-housing.csv')

In [ ]:

# 데이터 훑어보기

house
house.info() # 데이터의 개수, 결측치 및 데이터 타입 확인
house.describe() # 수치형 데이터의 기초 통계자료 확인


In [ ]:
# EDA
## 이상치 존재 여부 확인 및 처리 -> IQR 방식으로
##   -> IQR로 계산된 상한선과 하한선을 벗어난 데이터를 이상치로 처리하는 방법

Q1 = house.quantile(0.25) # 1분위 값
Q3 = house.quantile(0.75) # 3분위 값

IQR = Q3 - Q1 # IQR 계산

highest = Q3 + IQR * 1.5 # 상한선 계산
lowest = Q1 - IQR * 1.5 # 하한선 계산

## 아래와 같은 방식은 컬럼의 개수가 많아지면 힘들어질 수 있다.
total_rooms_high = highest['total_rooms']
total_rooms_low = lowest['total_rooms']
house['total_rooms'][(house['total_rooms'] > total_rooms_high) | (house['total_rooms'] < total_rooms_low)]


In [ ]:
## pandas의 apply를 이용하여 쉽게 결측치를 제거할 수 있음.
## apply -> 데이터프레임 컬럼/ 로우를 함수에 넘겨서 처리


## 로우의 이상치 여부 판별 함수
def is_not_outlier(row) :
  if row.sum() > 0 : # 이상치
    return False
  else : # 정상
    return True

tmp_df = (house > highest) | (house < lowest) # 데이터프레임의 값을 이상치 여부로 변환. (이상치는 False, 정상은 True)

normal_idx = tmp_df.apply(is_not_outlier, axis=1) # 이상치 여부 판별 함수를 적용하여 정상 데이터의 논리 인덱스만 걸러냄

house2 = house.loc[normal_idx] # 논리 인덱싱을 이용해 정상 데이터만 추출

In [ ]:
## 결측치 처리
house3 = house2.dropna()

## 결측치 처리 잘 되었는지 확인
house3.info()

In [ ]:
## 표준화를 진행하기 위한 수치형 데이터와 인코딩을 진행하기 위한 범주형 데이터를 분리
house3_num = house3.drop(['ocean_proximity','median_house_value'], axis=1)
house3_cate = house3[['ocean_proximity']]

## 정답데이터는 표준화하지 않는다.(할 필요가 없다.)
house3_target = house3['median_house_value']


In [ ]:
## 수치형 데이터
### 표준화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler() # 표준화 변환기 생성

ss.fit(house3_num) # 변환기에 수치형 데이터 세팅

s_house_num = ss.transform(house3_num) # 변환기의 transform 함수로 표준화 된 데이터 얻기

s_house_num_df = pd.DataFrame(s_house_num, columns=house3_num.columns.values) # 변환기는 결과를 numpy array로 돌려주므로 DataFrame으로 변환

s_house_num_df # 결과 확인

In [ ]:
## 범주형 데이터
house_cate_1hot = pd.get_dummies(house3_cate) # 범주형 데이터 원핫 인코딩

house_cate_1hot # 결과 확인

In [ ]:
## 범주형 데이터와 수치형 데이터 합치기
house_cate_1hot = house_cate_1hot.reset_index(drop=True) # 수치형 데이터의 데이터프레임 인덱스가 초기화 됐기 때문에 범주 데이터도 인덱스 초기화 해줘야 concat할 때 정상적으로 붙는다.
train = pd.concat([s_house_num_df, house_cate_1hot], axis=1) # 인덱스를 맞춘 수치형 데이터프레임과 범주형 데이터프레임을 좌우로 붙인다. 해당 데이터는 학습데이터로 완성된다.
target = house3_target # 학습데이터에 대한 정답 데이터

In [ ]:
## 공부용 데이터와 시험용 데이터로 나누기
from sklearn.model_selection import train_test_split

trd, tsd, trt, tst = train_test_split(train, target, train_size=0.8, random_state=42) # 학습용데이터는 꼭 2차원으로


In [ ]:
# 학습
## 선형회귀를 이용하여 모델 학습
from sklearn.linear_model import LinearRegression

lr2 = LinearRegression() # 선형회귀 모델 생성
lr2.fit(trd, trt) # 학습

In [ ]:
# 평가
## 이전 학습했던 lr 모델모다 lr2의 점수가 높아짐을 확인. 이전 모델 평가 결과는 바로 아래 셀에서 확인
print(lr2.score(trd, trt), lr2.score(tsd, tst))

In [ ]:
# 이전 median_income으로만 학습했을 때의 결과
from sklearn.linear_model import LinearRegression

lr = LinearRegression() # 선형회귀 모델 생성
lr.fit(trd, trt) # 학습

print(lr.score(trd, trt), lr.score(tsd, tst))

0.4769927345820525 0.45885918903846656


In [ ]:
## 이후 성능을 더 올리기 위해 할 수 있는 것
### 1. 데이터 추가
### 2. 추가적인 데이터 처리
### 3. 피처 엔지니어링(컬럼 조합, 다항식 적용)
### 4. 하이퍼 파라미터 튜닝(알고리즘에 관여하는 파라미터를 조정. 알고리즘 원리를 이해해야 함)
### 5. 위 방법으로도 성능이 올라가지 않으면 다른 알고리즘을 고민